In [1]:
# pip install openai

# imports
import os
from openai import AzureOpenAI
import pandas as pd

In [51]:
def get_client(endpoint = None, api_key = None, api_version = None): 
    import os
    from openai import AzureOpenAI
    if endpoint is None:
        endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    if api_key is None:
        api_key = os.getenv("AZURE_OPENAI_API_KEY")
    if api_version is None:
        api_version = os.getenv("AZURE_OPENAI_API_VERSION")

    client = AzureOpenAI(
        api_key=api_key,
        api_version=api_version, 
        azure_endpoint=endpoint
    )
    return client

In [47]:
client = get_client(api_version= "2023-05-15", api_key='dfsaf', endpoint='sfada')

In [70]:
def get_prompt(system_prompt = None, user_prompt = None, example = None):
    if system_prompt is None:
        system_prompt = "You are a helpful assistant."
    if user_prompt is None:
        user_prompt = "Summarize the following text."
    if example is None:
        example= ""
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": f"Question: {user_prompt}. Example(s): {example} \n"
        }]

In [71]:
x = get_prompt(system_prompt=None, user_prompt=None, example=None)

In [72]:
x[1]["content"] = x[1]["content"] + "fdsahklgjkfld;jgk;lfdjkl;a"

In [73]:
print(x[1]["content"])

Question: Summarize the following text.. Example(s):  
fdsahklgjkfld;jgk;lfdjkl;a


In [5]:
# initialize authentification credentials
endpoint = os.getenv("ENDPOINT_URL", "https://oai-test-ss.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-35-turbo")
api_key = os.getenv("AZURE_OPENAI_API_KEY", "f280112d275040608311f2920c9f1a28")
# os.environ["AZURE_OPENAI_API_KEY"] = 
# os.environ["DEPLOYMENT_NAME"] = SAS.SYMGET("genModelDeployment")
# os.environ["AZURE_OPENAI_ENDPOINT"] = SAS.SYMGET("azureOpenAIEndpoint")

In [60]:
x

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': 'Question: Summarize the following text.. Example(s): fdsahklgjkfld;jgk;lfdjkl;a'}]

In [39]:
# Obtain values from UI
system_prompt = "you are a helpful library assistant. For each description of a book, provide the genre of the book."
input_data = pd.read_csv("book_desc.csv")
doc_id = "title"
text_col = "description"
user_prompt = "{Question}What is the genre of this book? {Context} "
temperature = 0.7
output_table = pd.DataFrame(columns=[])
# input_data= SAS.symget("inputData")
# output_table = SAS.symget("outputTable")
# system_prompt = SAS.symget("system_prompt")
# user_prompt = SAS.symget("user_prompt")
# text_col = SAS.symget("textcol")
# temperature = SAS.symget(temperature)
# shots = SAS.symget("shots")

In [7]:
# initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version="2024-10-21", 
    azure_endpoint=endpoint
)


In [44]:
def call_llm(context): 
    # generate prompt
    chat_prompt = [
    {
        "role": "system",
        "content": system_prompt
    },
    {
        "role": "user",
        "content": user_prompt + context 
    }
    ]
    
    # call the llm 
    completion = client.chat.completions.create(
        model = deployment,
        messages = chat_prompt,
        temperature = temperature
    )  
    return completion.choices[0].message.content

In [45]:
input["response"]= input_data[text_col].apply(call_llm)

In [46]:
output_table.head()

,response
0,Fantasy.
1,Mystery/Thriller.
2,Romance/Contemporary Fiction


In [34]:
# Testing multicolumn assignment
def call_llm(row): 
    # generate prompt
    chat_prompt = [
    {
        "role": "system",
        "content": system_prompt
    },
    {
        "role": "user",
        "content": user_prompt + row[text_col] 
    }
    ]
    
    # call the llm 
    completion = client.chat.completions.create(
        model = deployment,
        messages = chat_prompt,
        temperature = temperature
    )  
    return completion.choices[0].message.content, completion.choices[0].finish_reason, completion.choices[0].content_filter_results['hate']['severity'], completion.choices[0].content_filter_results['self_harm']['severity'], completion.choices[0].content_filter_results['sexual']['severity'], completion.choices[0].content_filter_results['violence']['severity']


In [40]:
input_data[['response', 'finish_reason', 'response_hate', 'response_self_harm', 'response_sexual', 'response_violence']] = input_data.apply(call_llm, axis=1, result_type="expand")

In [41]:
input_data.head()

,title,author,description,response,finish_reason,response_hate,response_self_harm,response_sexual,response_violence
0,Onyx Storm,Rebecca Yarros,After nearly eighteen months at Basgiath War C...,The genre of this book is fantasy.,stop,safe,safe,safe,safe
1,Beautiful Ugly,Alice Feeney,Author Grady Green is having the worst best da...,The genre of this book is Mystery/Thriller.,stop,safe,safe,safe,safe
2,Great Big Beautiful Life,Emily Henry,Alice Scott is an eternal optimist still dream...,The genre of this book is romance.,stop,safe,safe,safe,safe


In [13]:
print(completion.to_json())

{
  "id": "chatcmpl-AtJE2DWzZ6YmFIu0PqQRGQrIKVuv5",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "role": "assistant"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1737745306,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 9,
    "prompt_tokens": 37,
    "total_tokens": 46
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "